# Exploratory Data Analysis

I will do a EDA to check necesary cleaning steps.

In [ ]:
# numerical libraries
import pandas as pd

# configs
pd.set_option('display.max_columns', None)

# paths
file_path = '../datasets/games.csv'

In [ ]:
# read information
games_df_raw = pd.read_csv(file_path)

# print basic info
games_df_raw.info()

In [ ]:
# we delete the 2025 season games as they are incomplete
games_df_selection = games_df_raw.drop(games_df_raw[games_df_raw['season'] == 2025].index).copy()

# we will asociate a number to each game type, asociated with its importance
# 1 = regular season
# 2 = wildcard playoff
# 3 = divisional round playoff
# 4 = conference championship
# 5 = super bowl
mapper = {'REG':1, 'WC':2, 'DIV':3, 'CON':4, 'SB':5}
games_df_selection['game_type_level'] = games_df_selection['game_type'].map(mapper)


'''


# selection of columns
games_df_raw[['game_id', 'season', 'game_type', 'week', 'gameday', 'weekday',
       'gametime', 'away_team', 'away_score', 'home_team', 'home_score',
       'location', 'result', 'total', 'overtime', 'old_game_id', 'gsis',
       'nfl_detail_id', 'pfr', 'pff', 'espn', 'ftn', 'away_rest', 'home_rest',
       'away_moneyline', 'home_moneyline', 'spread_line', 'away_spread_odds',
       'home_spread_odds', 'total_line', 'under_odds', 'over_odds', 'div_game',
       'roof', 'surface', 'temp', 'wind', 'away_qb_id', 'home_qb_id',
       'away_qb_name', 'home_qb_name', 'away_coach', 'home_coach', 'referee',
       'stadium_id', 'stadium']]
''';

games_df_selection['game_type_level'].value_counts(sort=False)

In [ ]:
games_df_raw['season'].value_counts(sort=False)